In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableAssign
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from services import create_chroma_client
from typing import Sequence

In [3]:
embeddings = OllamaEmbeddings(model="mistral")
db = Chroma(
    client=create_chroma_client(),
    embedding_function=embeddings,
    collection_name="news",
)

In [4]:
model = ChatOllama(
    model="openchat",
    temperature=1,
    streaming=True,
)

In [5]:
def buscar_noticias(query: dict) -> Sequence[str]:
    import time
    import datetime as dt

    fecha = query["fecha"].lower()
    if fecha == "ayer":
        fecha_dt = dt.datetime.today() - dt.timedelta(days=1)
    else:
        fecha_dt = dt.datetime.strptime(fecha, "%Y-%m-%d")
    fecha_unix = time.mktime(fecha_dt.timetuple())

    documents = db.similarity_search(
        query["tematica"],
        k=5,
        filter={"published": {"$gt": fecha_unix}},
    )

    if len(documents) > 0:
        return "- " + "\n- ".join([document.page_content for document in documents])
    return "Sin noticias relevantes"

In [6]:
template_tematica = """A partir de la pregunta del usuario, debes extraer la temática 
a la cual se refiere. Lo que importa es el sujeto, no adjetivos, fechas, etc.

Por ejemplo:
    - "qué pasó en Rio de Janeiro ayer?", la temática es "Rio de Janeiro"
    - "qué hiciste ayer en el centro de Rosario?", la temática es "centro de Rosario"
    - "cuantas plantas tiene tu casa?", la temática es "plantas de tu casa"

Solo debes responder el nombre de la temática ignorando el resto de la pregunta:

{input}
"""
prompt_tematica = ChatPromptTemplate.from_template(template_tematica)

chain_tematica = (
    RunnableParallel({'input': RunnablePassthrough()})
    | prompt_tematica
    | model
    | StrOutputParser()
    | (lambda x: x.strip())
)

chain_tematica.name = "Tematica"

In [7]:
template_fecha = """A partir de la pregunta del usuario,
debes extraer una fecha a la cual hace referencia.
Por ejemplo:
    - "qué vas a hacer mañana?", la fecha es "mañana"
    - "qué hiciste ayer?", la fecha es "ayer"
    - "qué hiciste el 1 de noviembre de 1990?", la fecha es "1990-11-01"
    
En caso de detectar una fecha especifica, devuelvela en formato "AAAA-MM-DD".
En caso de no estar seguro cuál es la fecha, asume "Ayer".

Solo debes responder la fecha que detectes:

{input}
"""
prompt_fecha = ChatPromptTemplate.from_template(template_fecha)

chain_fecha = (
    RunnableParallel({'input': RunnablePassthrough()})
    | prompt_fecha
    | model
    | StrOutputParser()
    | (lambda x: x.strip())
)

chain_fecha.name = "Fecha"

In [8]:
chain_noticias = (
    RunnableParallel(
        {
            "tematica": chain_tematica,
            "fecha": chain_fecha,
        }
    )
    | buscar_noticias
)

chain_noticias.name = "Noticias"

In [9]:
template = """A partir de la siguiente pregunta del usuario:

{pregunta}

Y las siguientes noticias que hablan de la temática del usuario:

{contexto}

Elabora una respuesta a la pregunta.
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "contexto": chain_noticias,
            "pregunta": RunnablePassthrough(),
        }
    )
    | prompt
    | model
    | StrOutputParser()
)

chain.name = "Resumen noticias"

In [10]:
INPUT = "Que paso ayer en gran hermano?"

In [11]:
print(chain.invoke(INPUT))

 Ayer en Gran Hermano hubo varias situaciones interesantes que causaron reacciones y discusiones entre los concursantes. El momento en particular que se ha referido el usuario no está claro, pero algunos de los eventos importantes fueron:

1. Valeria Mazza y Alejandro Gravier en la tapa de Caras: "Somos un equipo". Este encuentro entre dos conocidos de la industria del espectáculo generó mucha expectativa y conversación.

2. Beto Casella apuntó contra Luis Majul: lo acusó de copiar "Bendita TV". La disputa entre estos dos artistas se volvió viral en las redes sociales, creando un gran impacto público.

3. Gran Hermano: Furia dio detalles de su encuentro íntimo con Mauro. Este evento dentro del programa también generó mucha atención y conversación entre los seguidores de la serie.

4. Netflix: últimos días para ver un clásico de comedia y baile que supo ser furor. Se trata de una serie muy popular en el servicio de streaming, cuya programación está por finalizar, lo que ha llevado a los